# Example using TORRENTpy for a semi-distributed catchment

This notebook is an example of the use of the TORRENTpy Application Programming Interface (API) to simulate the hydrology and water solutes and particules transport in a lumped catchment. Example data files that can be used for this example are available in `examples/in/CatchmentLumpedName_OutletName` and `examples/in/CatchmentSemiDistributedName_OutletName` folders, for lumped and semi-distributed examples, respectively.

The first step is, of course, to import the Python package `torrentpy`.

In [ ]:
import torrentpy

## Set up

Then, the first object to instantiate is always from the class `Network`. It will create the node-link network itself, and collect all the information required for these.

In [ ]:
nw = torrentpy.Network(
    catchment='CatchmentSemiDistributedName', 
    outlet='OutletName', 
    input_folder='in/CatchmentSemiDistributedName_OutletName/', 
    output_folder='out/CatchmentSemiDistributedName_OutletName/',
    variable_h='q_h2o',
    variables_q=['c_no3', 'c_nh4', 'c_dph', 'c_pph', 'c_sed'],
    water_quality=True,
    verbose=True
)

An instance of the `TimeFrame` class is required. Be warned, the first four arguments need to be datetime.datetime objects from the package `datetime`. The arguments containing __data__ refer to the time information about the input data, the arguments containing __save__ refer to the desired time attributes of the output data, and the attributes containing __simu__ refer to the internal time information that will be used during simulation (typically the discretisation time step).

In [ ]:
tf = torrentpy.TimeFrame(
    dt_data_start=datetime.datetime.strptime('01/01/2008 09:00:00', '%d/%m/%Y %H:%M:%S'), 
    dt_data_end=datetime.datetime.strptime('31/12/2012 09:00:00', '%d/%m/%Y %H:%M:%S'), 
    dt_save_start=datetime.datetime.strptime('01/01/2009 09:00:00', '%d/%m/%Y %H:%M:%S'), 
    dt_save_end=datetime.datetime.strptime('31/12/2010 09:00:00', '%d/%m/%Y %H:%M:%S'),
    data_increment_in_minutes=1440, 
    save_increment_in_minutes=1440, 
    simu_increment_in_minutes=60,
    expected_simu_slice_length=2000, 
    warm_up_in_days=365
)

In order to get access to the models already available in TORRENTpy (and appending your own ones), an instance of `KnowledgeBase` has to be created.

In [ ]:
kb = torrentpy.KnowledgeBase()

It is finally time to bring in the input data. TORRENTpy allows for CSV files and NetCDF files to be used using the argument `in_format`. Then, __meteorological__ data are required, __contamination__ data are required only if water quality simulation is desired. TORRENTpy can disaggregate input datasets if required, this is why for each type of data (meteorological and contamination) it is required to distinguish between __cumulative__ and __average__ data. If disaggregation is required: cumulative data will be disaggregated using a simple assumption of a uniform distribution in the past; average data will be disaggregate by replicating the values in the past.

In [ ]:
db = torrentpy.DataBase(
    nw, tf, kb,
    in_format='csv',
    meteo_cumulative=['rain', 'peva'],
    meteo_average=['airt', 'soit'],
    contamination_cumulative=['m_no3', 'm_nh4', 'm_p_ino', 'm_p_org'],
    contamination_average=[]
)

The last step is to assign models to the links in the network. Assigning models for __catchments__ and __rivers__ for each link, while __lakes__ models are only required if lakes are present in the network. Hydrological models are mandatory (___h__) while water quality models (___q__) are only compulsory if water quality is set to `True` in the instance of `Network`. In this example, the rainfall-runoff model `SMART` and the water quality components of the `INCA` are used. These models come as built-in models in TORRENTpy.

In [ ]:
nw.set_links_models(
    kb,
    catchment_h='SMART', river_h='SMART',
    catchment_q='INCA', river_q='INCA'
)

## Simulate

Everything is now in place for the simulation to be launched, one last instruction and the simulation will be underway. Once again, TORRENTpy gives the choice for the file format for the output data, choose between *csv* and *netcdf*.

In [ ]:
nw.simulate(
    db, tf, out_format=out_fmt
)